### Gewöhnliche Differentialgleichungen

Das Einzelpendelproblem kann als gewöhnliche Differentialgleichung der Form betrachtet werden:

\begin{align}
  \frac{d \vec{u}(t)}{dt} & = \vec{f}(t,\vec{u}(t))
\end{align}

wobei $\vec{u}(t) (= y(t))$ der Zustandsvektor zur Zeit $t$ ist, bestehend aus Position $x(t)$ und Geschwindigkeit $v(t)$. 

Im Allgemeinen können gewöhnliche Differentialgleichungen nicht analytisch gelöst werden (manchmal funktioniert es).

$\leadsto$ Wir benötigen numerische Lösungsverfahren

# Simulation des Einzelpendels


\begin{align*}
 \varphi(t):&  \quad \text{Auslenkung aus der Gleichgewichtslage} && \quad [\varphi] = 1~\rm{rad}\\
 v(t):&  \quad \text{Bahngeschwindigkeit des Pendels} && \quad [v] = 1~\rm{m/s} \\
 \\
 g : & \quad \text{Gravitationskonstante} && \quad [g] = 1~\rm{m/s^2}  \\
 l : & \quad \text{Länge des Pendels} && \quad [l] = 1~\rm{m}  \\
 k =\frac{b}{m}: & \quad \text{Dämpfungskonstante des Pendels} && \quad [k] = 1 / \rm{s}
\end{align*}

\begin{align*}
    \frac{d \varphi(t)}{dt} & = v(t) / l \\
    \frac{d v(t)}{dt} & = -g\sin(\varphi(t)) - k v(t)
\end{align*}

## Anwendung der expliziten Eulermethode:

#### Formulierung als allgemeine gewöhnliche Differenzialgleichung (ODE):

\begin{align}
  \frac{d \vec{u}(t)}{dt} & = \vec{f}(\vec{u}(t))
\end{align}
mit
$$
\vec{u}(t) = 
\left( 
\begin{array}{c}
\varphi(t) \\ v(t)
\end{array}
\right)
$$
und
$$
\vec{f}(\vec{u}) = \vec{f}(\varphi,v) = \left( 
\begin{array}{c}
 v / l\\ 
 - g \sin(\varphi) - k v
\end{array}
\right)
= \left( 
\begin{array}{c}
 \vec{u}_2 / l  \\
 - g \sin(\vec{u}_1) - k \vec{u}_2
\end{array}
\right)
$$


Dadurch wird die allgemeine explizite Eulermethode
\begin{align}
  \vec{u}^{n+1} & = \vec{u}^{n} + \Delta t \cdot \vec{f}(\vec{u}^n)
\end{align}
zu folgender Regel:
\begin{align}
  \varphi^{n+1} & = \varphi^{n} + \Delta t \cdot v^{n} / l \\
  v^{n+1} & = v^{n} + \Delta t \cdot ( - g \sin(\varphi^n) - k v^n  )
\end{align}

#### Anfangswerte / Materialparameter

* Anfangswerte:

In [ ]:
initialvals = { "phi" : 1.0, "v" : 0.0}

* Materialparameter:

In [ ]:
matparam =  { "g" : 1.0, "l" : 1.0, "k" : 0.0, "m" : 1.0}

### Ein-Schritt-Eulermethode:

Aufgabe:
 * Berechnen Sie für den Zustand $\varphi^n, v^n$ (d.h. $\vec{u}^n$) den nächsten Schritt $\varphi^{n+1}, v^{n+1}$ (d.h. $\vec{u}^{n+1}$).

In [ ]:
from math import sin, cos, pi
def OneStep_ExplicitEuler(u, matparam, simparam):
    phi0, v0 = u
    u[0] += simparam["dt"] * v0 / matparam["l"] 
    u[1] -= simparam["dt"] * ( matparam["g"] * sin(phi0) + matparam["k"] * v0)

### Zeitschrittverfahren:

In [ ]:
from numpy import array
from plot import plot
import matplotlib
%matplotlib inline
def ODESolve(initialvals, matparam, simparam):
    timehistory = []
    poshistory = []
    velhistory = []
    u = array([initialvals["phi"],initialvals["v"]],dtype=float)
    t = simparam["t0"]
    timehistory.append(t)
    poshistory.append(u[0])
    velhistory.append(u[1])
    dt = simparam["dt"]
    while t < simparam["tend"]:
        simparam["method"](u,matparam,simparam)
        t = t + dt
        timehistory.append(t)
        poshistory.append(u[0])
        velhistory.append(u[1])
    plot(timehistory,poshistory,velhistory)
    return u

### Berechnung der Energie

In [ ]:
def Energy(matparam,phi,v):
    return matparam["g"] * matparam["l"] * matparam["m"] * (1-cos(phi)) + 0.5 * matparam["m"] * v**2

### Simulation mit der expliziten Eulermethode

In [ ]:
simparam = { "dt" : 0.1, "t0" : 0, "tend" : 4*pi, "method" : OneStep_ExplicitEuler}
phi0, v0 = initialvals["phi"], initialvals["v"]
u_final = ODESolve(initialvals, matparam, simparam)
phi, v = u_final
print("Initial angle: {:8.3e}, Initial velocity: {:8.3e}, Initial energy: {:8.3e}".format(phi0, v0, Energy(matparam,phi0,v0)))
print("Final angle:     {:8.3e}, Final velocity:     {:8.3e}, Final energy:     {:8.3e}".format( phi,  v, Energy(matparam, phi, v)))

#### Aufgabe:
  * Simuliere für verschiedene Parameter $g,~l$ und $k$ über mehrere Schwingungsperioden.
  * Verringere immer $\Delta t$, bis sich das Simulationsergebnis nicht mehr sichtbar verändert.
  * Was beobachten Sie?

#### Aufgabe:
  * Betrachte auch Anfangswinkel von $\pi$ oder verschiedene Anfangsgeschwindigkeiten
  * Warum werden nach $4\pi$ keine zwei vollen Schwingungsperioden abgeschlossen? 

#### Aufgabe: 
  * Betrachte $k = 0$. 
  * Welche Gesamtenergie bleibt nach einigen Schwingungen im System?
  * Welche Gesamtenergie sollte theoretisch im System bleiben?
  * Wie hängt der Energiefehler von der Anzahl der Schwingungen und von der Schrittweite $\Delta t$ ab?

### Die symplektische Eulermethode

Die explizite Eulermethode wird für Mehrkörperprobleme häufig folgendermaßen modifiziert, um ein besseres Langzeitverhalten zu erzielen:

\begin{align}
  v^{n+1} & = v^{n} + \Delta t \cdot ( - g \sin(\varphi^{n}) - k v^n  ) \\
  \varphi^{n+1} & = \varphi^{n} + \Delta t \cdot v^{n+1} / l
\end{align}

Der einzige Unterschied zur expliziten Eulermethode ist die Benutzung des neuen $v$ für die Aktualisierung von $\varphi$.

In [ ]:
def OneStep_SymplecticEuler(u, matparam, simparam):
    phi0, v0 = u
    u[1] -= simparam["dt"] * ( matparam["g"] * sin(phi0) + matparam["k"] * v0)
    u[0] += simparam["dt"] * u[1]/matparam["l"]

#### Simulation mit der symplektischen Eulermethode

In [ ]:
simparam = { "dt" : 0.01, "t0" : 0, "tend" : 4*pi, "method" : OneStep_SymplecticEuler}
phi0, v0 = initialvals["phi"], initialvals["v"]
u_final = ODESolve(initialvals, matparam, simparam)
phi, v = u_final
print("Initial angle: {:8.3e}, Initial velocity: {:8.3e}, Initial energy: {:8.3e}".format(phi0, v0, Energy(matparam,phi0,v0)))
print("Final angle:     {:8.3e}, Final velocity:     {:8.3e}, Final energy:     {:8.3e}".format( phi,  v, Energy(matparam, phi, v)))

#### Aufgabe (wie oben):
  * Betrachte $k = 0$.
  * Welche Gesamtenergie bleibt nach einigen Schwingungen im System?
  * Welche Gesamtenergie sollte theoretisch im System bleiben?
  * Wie hängt der Energiefehler von der Anzahl der Schwingungen und von der Schrittweite $\Delta t$ ab?

#### Aufgabe: 
  * Implementiere das verbesserte Euler-Verfahren / Heun-Verfahren durch Anlegen einer neuen Funktion
  `OneStep_Heun(...)`, ähnlich wie `OneStep_ExplicitEuler(...)`. Vervollständigen Sie dazu einfach den untenstehenden Code und entfernen Sie die Zeile `raise Exception("Heun not yet implemented")`.
  * Vergleiche das Heun-Verfahren mit den oben behandelten Verfahren. Was beobachten Sie (Genauigkeit/Aufwand/Langzeitverhalten)?

In [ ]:
def OneStep_Heun(u, matparam, simparam):
    raise Exception("Heun not yet implemented")
    phi0, v0 = u
    uhalf0 = ...
    uhalf1 = ... 
    u[0] = ...
    u[1] = ...   

#### Simulation mit dem Heun-Verfahren

In [ ]:
simparam = { "dt" : 0.1, "t0" : 0, "tend" : 4*pi, "method" : OneStep_Heun}
phi0, v0 = initialvals["phi"], initialvals["v"]
u_final = ODESolve(initialvals, matparam, simparam)
phi, v = u_final
print("Initial angle: {:8.3e}, Initial velocity: {:8.3e}, Initial energy: {:8.3e}".format(phi0, v0, Energy(matparam,phi0,v0)))
print("Final angle:     {:8.3e}, Final velocity:     {:8.3e}, Final energy:     {:8.3e}".format( phi,  v, Energy(matparam, phi, v)))

#### Zusatzaufgabe: 
  * Implementiere ein weiteres Verfahren
  `OneStep_StoermerVerlet(...)` ähnlich wie `OneStep_ExplicitEuler(...)`. Vervollständigen Sie dazu einfach den untenstehenden Codeblock und entfernen Sie die Zeile `raise Exception("Heun not yet implemented")`.
  Das Störmer-Verlet-Verfahren ist wie folgt definiert:
  \begin{align}
  \varphi^{n+\frac12} & = \varphi^{n} + \Delta t/2 ~ v^{n} / l \\
  v^{n+\frac12} & = v^{n} + \Delta t/2 ~  ( - g \sin(\varphi^{n+\frac12}) - k v^n  ) \\
  v^{n+1} & = v^{n+\frac12} + \Delta t/2 ~  ( - g \sin(\varphi^{n+\frac12}) - k v^{n+\frac12} ) \\
  \varphi^{n+1} & = \varphi^{n+\frac12} + \Delta t/2 ~ v^{n+1} / l \\
\end{align}
    
  * Welche Vorteile beobachtest du für dieses Verfahren?

In [ ]:
def OneStep_StoermerVerlet(u, matparam, simparam):
    raise Exception("Heun not yet implemented")
    phi0, v0 = u
    u[0] = ...
    u[1] = ...

#### Simulation mit dem Störmer-Verlet-Verfahren

In [ ]:
matparam =  { "g" : 1, "l" : 1, "k" : 0.0, "m" : 1.0}
simparam = { "dt" : 0.1, "t0" : 0, "tend" : 4*pi, "method" : OneStep_StoermerVerlet}
phi0, v0 = initialvals["phi"], initialvals["v"]
u_final = ODESolve(initialvals, matparam, simparam)
phi, v = u_final
print("Initial angle: {:8.3e}, Initial velocity: {:8.3e}, Initial energy: {:8.3e}".format(phi0, v0, Energy(matparam,phi0,v0)))
print("Final angle:     {:8.3e}, Final velocity:     {:8.3e}, Final energy:     {:8.3e}".format( phi,  v, Energy(matparam, phi, v)))